# Automated Occupation Classification and Mapping using Semantic Similarity and Zero-shot Learning
## IMD1107 - Natural Language Processing
### [Dr. Elias Jacob de Menezes Neto](htttps://docente.ufrn.br/elias.jacob)

## Keypoints
- **Automated Occupation Classification**: The task involves automatically classifying occupation names into a standardized set of professions using advanced NLP and ML techniques.

- **Challenges**: Key challenges include semantic variability, ambiguity, lack of labeled data, the need for generalization, and efficiency.

- **IBGE Classification**: The Brazilian Institute of Geography and Statistics (IBGE) provides a standardized classification of professions used as a reference.

- **Approach**: The approach includes information retrieval, text classification, data preprocessing, transfer learning, semantic similarity analysis, zero-shot learning, nearest
neighbor search, and domain-specific adaptation.

- **Embedding Models**: Two approaches for embedding models are discussed: an open-source approach using BGE-M3 and a commercial approach using OpenAI's models.

- **Reranking Models**: Reranking models (cross-encoders) are used to refine search results and improve accuracy by considering query-specific context.

- **Zero-shot Learning**: This technique allows the model to classify occupation names into professions without explicit training on those names, using the model's generalization capabilities.

- **Cost Considerations**: The cost of using different models is analyzed, with a focus on balancing performance and budget constraints.


## Learning Goals

By the end of this class, you will be able to:

1) **Define** the key challenges in automated occupation classification, including semantic variability, ambiguity, and the lack of labeled data, and **explain** why these challenges necessitate advanced NLP techniques.

2) **Implement** a system for occupation classification using semantic similarity by **applying** embedding models (like BGE-M3 or OpenAI embeddings) and **calculating** cosine similarity to identify the most relevant standardized professions for a given occupation name.

3) **Utilize** zero-shot learning with Large Language Models (LLMs) to **classify** occupation names into predefined categories without explicit training data for each category, and **explain** the advantages of this approach for handling unseen occupation titles.

4) **Evaluate** the impact of reranking models (cross-encoders) on improving the accuracy of occupation classification and **compare** their performance and computational cost with bi-encoder models used for initial retrieval.

5) **Compare and contrast** open-source (e.g., Ollama with Llama 3.1 and BGE-M3) and commercial (e.g., OpenAI) approaches for embedding and language models in the context of occupation classification, considering factors such as cost, performance, customization, and data privacy.


### Our Problem

In many practical applications within Natural Language Processing (NLP) and Machine Learning (ML), it is necessary to classify diverse text inputs into standardized categories. In this context, the challenge is to assign occupation names entered by users to a predefined set of professions based on the Brazilian Institute of Geography and Statistics (IBGE) classification, which comprises 2557 professions. This task is particularly important for the Public Defender's Office at Rio Grande do Norte (DPE/RN), where accurate and consistent categorization is essential for statistical analysis and reporting.

> **Key Objective:** Map each user-provided occupation name to the closest matching standardized profession using advanced computational techniques.

### Challenges in Occupation Classification

There are several factors that contribute to the complexity of this classification task:

1. **Semantic Variability**  
   - **Issue:** The wording and structure of occupation names vary widely.  
   - **Impact:** This variability makes it difficult to achieve a straightforward match with standardized classifications.

2. **Ambiguity and Overlap**  
   - **Issue:** Many occupation names can correspond to more than one standardized profession.  
   - **Impact:** The system must carefully determine which profession most accurately represents the input, taking into account overlapping and ambiguous terms.

3. **Lack of Labeled Data**  
   - **Issue:** There is no abundant source of explicitly labeled pairs of occupation names and professions.  
   - **Impact:** Conventional supervised learning methods are less effective, which necessitates the use of alternative approaches such as zero-shot learning.

4. **Generalization and Adaptation**  
   - **Issue:** The classification system must perform accurately on both seen and unseen occupation names.  
   - **Impact:** The method used must be reliable enough to adapt to new data while maintaining reliability.

5. **Efficiency and Accuracy**  
   - **Issue:** The system is expected to process a large volume of occupation names quickly without sacrificing accuracy.  
   - **Impact:** Balancing speed and precision makes the design and implementation of the system both challenging and critical.


### Our Approach

To address the challenges mentioned above, a multi-step approach is employed that integrates several computational techniques. The key components of the approach are as follows:

#### 1. Information Retrieval and Text Classification
- **Purpose:** Identify the most relevant profession from the standardized list based on each occupation name.
- **Method:** Combine search methods with classification algorithms to narrow down the candidates from the IBGE classification.

#### 2. Data Preprocessing and Normalization
- **Purpose:** Clean and standardize raw input data.
- **Method:** Remove noise and inconsistencies to improve the quality of the classification input.
- **Example:** Converting text to lowercase, removing special characters, and standardizing abbreviations.

#### 3. Transfer Learning
- **Purpose:** Use pre-trained language models to reduce the dependence on large amounts of task-specific labeled data.
- **Method:** Apply models that have been trained on general language tasks to understand the contextual meaning of occupation names.
- **Benefit:** This technique improves the system's ability to recognize and classify unseen occupation names.

#### 4. Semantic Similarity Analysis
- **Purpose:** Measure the semantic closeness between occupation names and standardized professions.
- **Method:** Represent occupation names and professions using word embeddings.

#### 5. Zero-shot Learning
- **Purpose:** Classify occupation names into categories without explicit training on each profession.
- **Method:** Use models that can infer the relationship between new text inputs and existing categories based on general language understanding.
- **Outcome:** Enables the system to handle previously unseen occupation names effectively.

#### 6. Nearest Neighbor Search in Embedding Space
- **Purpose:** Find the most similar standardized professions for each occupation name by analyzing their vector representations.
- **Method:** Visualize the matching process as a k-nearest neighbors (k-NN) search in the high-dimensional embedding space.
- **Mathematical Insight:**  
  If $\mathbf{x}_i$ represents the embedding of an occupation name and $\{\mathbf{y}_j\}$ represents the embeddings of standardized professions, then the closest match is given by:
  $$
  \text{match} = \underset{j}{\arg\min}\ d(\mathbf{x}_i, \mathbf{y}_j)
  $$
  where $d(\mathbf{x}_i, \mathbf{y}_j)$ might be the Euclidean distance or $1 - \text{cos\_sim}(\mathbf{x}_i, \mathbf{y}_j)$.


In [1]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

from multiprocessing import cpu_count
from typing import Any, Literal

import joblib
import numpy as np
import pandas as pd
import torch
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from pydantic import BaseModel, Field
from sklearn.metrics.pairwise import cosine_similarity
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)

# Load environment variables from a .env file. This is necessary to access the OPENAI_API_KEY environment variable.
load_dotenv()

True

In [2]:
# Load the official professions data from a CSV file into a DataFrame
# This file contains professions with CBO (Brazilian Classification of Occupations) codes
official_data = pd.read_csv("data/profissoes_com_cbo.csv")

# Load the unofficial professions data from a CSV file into a DataFrame
# This file contains professions without CBO codes
unofficial_data = pd.read_csv("data/profissoes_sem_cbo.csv")

In [3]:
official_data

,id,codigo,nome
0,1,10105,Oficial general da aeronáutica
1,2,10110,Oficial general do exército
2,3,10115,Oficial general da marinha
3,4,10215,Oficial da marinha
4,5,10205,Oficial da aeronáutica
...,...,...,...
2552,2553,992205,Encarregado geral de operações de conservação ...
2553,2554,992220,Pedreiro de conservação de vias permanentes (e...
2554,2555,992215,Operador de ceifadeira na conservação de vias ...
2555,2556,992210,Encarregado de equipe de conservação de vias p...


In [4]:
unofficial_data

,id,codigo,nome
0,2558,NaN,ADMINISTRADOR DE SISTEMAS OPERACIONAIS
1,2559,NaN,ANALISTA DE INFORMAÇÕES (PESQUISADOR DE INFORM...
2,2560,NaN,EMPREGADO DOMÉSTICO DIARISTA
3,2561,NaN,APOSENTADO
4,2562,NaN,AGRICULTORA
...,...,...,...
8246,10862,NaN,ESTUDANTE - 8 ANOS
8247,10863,NaN,COORDENADOR DE VENDAS
8248,10864,NaN,FARMACÊUTICA - AUXÍLIO DOENÇA
8249,10865,NaN,AUXILIAR DE SERVENTE


In [5]:
# Rename columns in the official_data DataFrame for better readability and consistency
# 'codigo' is renamed to 'code' and 'nome' is renamed to 'text'
official_data.rename({"codigo": "code", "nome": "text"}, axis=1, inplace=True)

# Rename columns in the unofficial_data DataFrame for better readability and consistency
# 'codigo' is renamed to 'code' and 'nome' is renamed to 'text'
unofficial_data.rename({"codigo": "code", "nome": "text"}, axis=1, inplace=True)

# Display the official_data DataFrame to verify the changes
official_data

,id,code,text
0,1,10105,Oficial general da aeronáutica
1,2,10110,Oficial general do exército
2,3,10115,Oficial general da marinha
3,4,10215,Oficial da marinha
4,5,10205,Oficial da aeronáutica
...,...,...,...
2552,2553,992205,Encarregado geral de operações de conservação ...
2553,2554,992220,Pedreiro de conservação de vias permanentes (e...
2554,2555,992215,Operador de ceifadeira na conservação de vias ...
2555,2556,992210,Encarregado de equipe de conservação de vias p...


In [6]:
unofficial_data

,id,code,text
0,2558,NaN,ADMINISTRADOR DE SISTEMAS OPERACIONAIS
1,2559,NaN,ANALISTA DE INFORMAÇÕES (PESQUISADOR DE INFORM...
2,2560,NaN,EMPREGADO DOMÉSTICO DIARISTA
3,2561,NaN,APOSENTADO
4,2562,NaN,AGRICULTORA
...,...,...,...
8246,10862,NaN,ESTUDANTE - 8 ANOS
8247,10863,NaN,COORDENADOR DE VENDAS
8248,10864,NaN,FARMACÊUTICA - AUXÍLIO DOENÇA
8249,10865,NaN,AUXILIAR DE SERVENTE


In [7]:
# Check for duplicate entries in the 'text' column of the official_data DataFrame
# Drop duplicates based on the 'text' column and get the shape of the resulting DataFrame
# Compare the shape of the DataFrame with duplicates removed to the original DataFrame shape
# This helps to confirm that there are no duplicate entries in the 'text' column

official_data.drop_duplicates(
    subset="text"
).shape, official_data.shape  # There are no duplicates

((2557, 3), (2557, 3))

In [8]:
# Convert all text entries in the 'text' column of the unofficial_data DataFrame to lowercase
# This ensures uniformity in text data, making it easier to compare and process
unofficial_data["text"] = unofficial_data["text"].str.lower()

official_data["text"] = official_data["text"].str.lower()

In [9]:
# Split the 'text' column of the unofficial_data DataFrame at the first occurrence of the '-' character
# Create a new column 'text_split_1' containing the part before the '-' character, with leading and trailing spaces removed
unofficial_data["text_split_1"] = unofficial_data["text"].apply(
    lambda x: x.split("-")[0].strip()
)

# Do the same for the official_data DataFrame
# If there is no '-' character in the text, assign None to 'text_split_2'
unofficial_data["text_split_2"] = unofficial_data["text"].apply(
    lambda x: x.split("-")[-1].strip() if len(x.split("-")) > 1 else None
)

In [10]:
unofficial_data["text_split_1"].value_counts().head(50)

text_split_1
autonomo                       141
do lar                         114
autônomo                        93
estudante                       84
autonoma                        82
desempregado                    74
agricultora                     72
desempregada                    69
aposentado                      45
autônoma                        42
aposentada                      37
agricultor                      34
criança                         29
asg                             25
motorista                       25
pedreiro                        24
vendedora                       20
professora                      20
comerciante                     19
costureira                      17
dona de casa                    17
vendedor                        16
faz bico                        15
manicure                        14
diarista                        13
vendedor ambulante              12
motorista de aplicativo         12
empregada doméstica             12
vigilan

In [11]:
unofficial_data["text_split_2"].value_counts().head(
    50
)  # Most data after the - is not useful

text_split_2
desempregado                   131
autonomo                       127
desempregada                   110
autônomo                        87
auxilio brasil                  70
autonoma                        62
bolsa família                   57
bolsa familia                   45
registrado em carteira          44
bpc                             43
autônoma                        39
aposentado                      38
aposentada                      31
                                23
auxilio doença                  21
auxílio brasil                  20
pensionista                     17
clt                             17
carteira assinada               16
mei                             15
seguro desemprego               14
bicos                           13
do lar                          12
diarista                        11
aux. brasil                     10
cargo comissionado              10
asg                             10
comerciante                     10
receben

In [12]:
unofficial_data["text_split_2"].value_counts().head(50).index.tolist()

['desempregado',
 'autonomo',
 'desempregada',
 'autônomo',
 'auxilio brasil',
 'autonoma',
 'bolsa família',
 'bolsa familia',
 'registrado em carteira',
 'bpc',
 'autônoma',
 'aposentado',
 'aposentada',
 '',
 'auxilio doença',
 'auxílio brasil',
 'pensionista',
 'clt',
 'carteira assinada',
 'mei',
 'seguro desemprego',
 'bicos',
 'do lar',
 'diarista',
 'aux. brasil',
 'cargo comissionado',
 'asg',
 'comerciante',
 'recebendo seguro desemprego',
 'estudante',
 'loas',
 'invalidez',
 'informal',
 'vendas',
 'pedreiro',
 'manicure',
 'afastado',
 'agricultora',
 'servidora pública',
 'uber',
 'vigilante',
 'recebe bolsa familia',
 'pensão por morte',
 'supermercado',
 'inss',
 'afastada',
 'motorista',
 'recebe bolsa família',
 'motoboy',
 'segurança']

In [13]:
unofficial_data["text_split_1"].value_counts().head(50).index.tolist()

['autonomo',
 'do lar',
 'autônomo',
 'estudante',
 'autonoma',
 'desempregado',
 'agricultora',
 'desempregada',
 'aposentado',
 'autônoma',
 'aposentada',
 'agricultor',
 'criança',
 'asg',
 'motorista',
 'pedreiro',
 'vendedora',
 'professora',
 'comerciante',
 'costureira',
 'dona de casa',
 'vendedor',
 'faz bico',
 'manicure',
 'diarista',
 'vendedor ambulante',
 'motorista de aplicativo',
 'empregada doméstica',
 'vigilante',
 'auxiliar de cozinha',
 'feirante',
 'pedagoga',
 'pintor',
 'mei',
 'mecânico',
 'empresário',
 'serviços gerais',
 'bpc',
 'servente de pedreiro',
 'servente',
 'autonôma',
 'servidora pública',
 'auxiliar de serviços gerais',
 'mecanico',
 'cabeleireira',
 'atendente',
 'balconista',
 'assistente social',
 'garçonete',
 'eletricista']

In [14]:
unemployed = [
    "loas",
    "auxílio brasil",
    "aux. brasil",
    "afastado",
    "seguro desemprego",
    "pensão por morte",
    "bpc",
    "aposentada",
    "recebendo seguro desemprego",
    "dona de casa",
    "desempregado",
    "do lar",
    "estudante",
    "criança",
    "aposentado",
    "auxilio brasil",
    "desempregada",
    "recebe bolsa família",
    "pensionista",
    "bolsa família",
    "faz bico",
    "invalidez",
    "inss",
    "recebe bolsa familia",
    "bolsa familia",
    "afastada",
    "auxilio doença",
]


self_employed = [
    "autônoma",
    "autônomo",
    "autonôma",
    "informal",
    "autonomo",
    "autonoma",
]

In [15]:
def replace_unemployed(text: str) -> str:
    for word in unemployed:
        if word in text:
            return "desempregado"
    return text


def replace_self_employed(text: str) -> str:
    for word in self_employed:
        if word in text:
            return "autônomo"
    return text

In [16]:
unofficial_data["text_split_1"].apply(replace_unemployed).apply(
    replace_self_employed
).value_counts().head(50)

text_split_1
desempregado                   1823
autônomo                        856
agricultora                      72
agricultor                       34
asg                              25
motorista                        25
pedreiro                         24
vendedora                        20
professora                       20
comerciante                      19
costureira                       17
vendedor                         16
manicure                         14
diarista                         13
vendedor ambulante               12
empregada doméstica              12
motorista de aplicativo          12
auxiliar de cozinha              11
vigilante                        11
feirante                         11
mei                              10
pedagoga                         10
pintor                           10
mecânico                         10
auxiliar de serviços gerais       9
mecanico                          9
serviços gerais                   9
servente       

In [17]:
unofficial_data["text_split_1"] = (
    unofficial_data["text_split_1"]
    .apply(replace_unemployed)
    .apply(replace_self_employed)
)

In [18]:
unofficial_data["text_split_1"] = unofficial_data["text_split_1"].apply(
    lambda x: str(x)
)

In [19]:
unofficial_data_dedup = unofficial_data.drop_duplicates(subset="text_split_1")
unofficial_data_dedup

,id,code,text,text_split_1,text_split_2
0,2558,NaN,administrador de sistemas operacionais,administrador de sistemas operacionais,None
1,2559,NaN,analista de informações (pesquisador de inform...,analista de informações (pesquisador de inform...,None
2,2560,NaN,empregado doméstico diarista,empregado doméstico diarista,None
3,2561,NaN,aposentado,desempregado,None
4,2562,NaN,agricultora,agricultora,None
...,...,...,...,...,...
8241,10857,NaN,pessoa com deficiência,pessoa com deficiência,None
8243,10859,NaN,agricultor de subsistência,agricultor de subsistência,None
8245,10861,NaN,técnico de segurança eletrônica,técnico de segurança eletrônica,None
8247,10863,NaN,coordenador de vendas,coordenador de vendas,None


In [20]:
official_data["text"] = official_data["text"].apply(lambda x: str(x))

official_data = official_data.query("text.str.len() >= 5")
unofficial_data_dedup = unofficial_data_dedup.query("text_split_1.str.len() >= 5")

In [21]:
official_occupations = official_data["text"].tolist()

unofficial_data_dedup = unofficial_data_dedup[
    ~unofficial_data_dedup.text_split_1.isin(official_occupations)
]

unofficial_data_dedup

,id,code,text,text_split_1,text_split_2
3,2561,NaN,aposentado,desempregado,None
4,2562,NaN,agricultora,agricultora,None
6,2564,NaN,auxiliar de creche,auxiliar de creche,None
7,2565,NaN,tecnico judiciário,tecnico judiciário,None
13,2571,NaN,técnico em enfermágem,técnico em enfermágem,None
...,...,...,...,...,...
8241,10857,NaN,pessoa com deficiência,pessoa com deficiência,None
8243,10859,NaN,agricultor de subsistência,agricultor de subsistência,None
8245,10861,NaN,técnico de segurança eletrônica,técnico de segurança eletrônica,None
8247,10863,NaN,coordenador de vendas,coordenador de vendas,None


In [22]:
# Convert the 'text_split_1' column of the unofficial_data_dedup DataFrame to a list
# This list contains the unique unofficial occupation names
unofficial_occupations = unofficial_data_dedup["text_split_1"].tolist()

# Convert the 'text' column of the official_data DataFrame to a list
# This list contains the official occupation names
official_occupations = official_data["text"].tolist()

# Add 'desempregado' (unemployed) to the list of official occupations
# This ensures that the list includes this common status
official_occupations.append("desempregado")

# Add 'aposentado' (retired) to the list of official occupations
# This ensures that the list includes this common status
official_occupations.append("aposentado")

# Add 'autônomo' (self-employed) to the list of official occupations
# This ensures that the list includes this common status
official_occupations.append("autônomo")

In [23]:
len(official_occupations)

2558

## Approaches to Embedding and Language Models

In this section, we'll explore two distinct approaches for executing our embedding and language models:

### 1. Open-Source Approach (Free)

This method leverages leading-edge open-source models:

- **Embedding Model**: [BGE-M3](https://huggingface.co/BAAI/bge-m3)
    - BGE-M3 is a advanced embedding model known for its efficiency and performance in generating high-quality text representations. It works well across various text lengths and domains.
    - It's particularly well-suited for tasks requiring semantic understanding and similarity comparisons.

- **Language Model (LLM)**: [Llama 3.1](https://llama.meta.com/)
    - Llama 3.1 is an advanced open-source large language model developed by Meta AI.
    - It offers impressive natural language processing capabilities while being freely accessible for research and development.

**Key Advantages**:
- Cost-effective solution for resource-constrained projects
- Full control over model deployment and customization
- Potential for on-premise use, addressing data privacy concerns

**Considerations**:
- May require more computational resources and technical expertise to apply
- Performance might not match commercial alternatives in some scenarios



### 2. Commercial Approach (Paid)

This approach uses OpenAI's powerful, proprietary models:

- **Embedding Model**: OpenAI Embedding Model
    - Known for producing high-quality embeddings across numerous applications
    - Regularly updated to incorporate the latest advancements in natural language processing

- **Language Model (LLM)**: OpenAI LLMs
    - latest performance in various natural language tasks
    - Continuously improved and expanded with new capabilities

**Key Advantages**:
- advanced performance with minimal setup required
- Expandable solution backed by stable infrastructure
- Regular updates and improvements from OpenAI

**Considerations**:
- Associated costs based on usage
- Potential limitations on customization and fine-tuning
- Dependency on external API and internet connectivity



### Choosing the Right Approach

The selection between these approaches depends on several factors:

- **Budget**: The open-source approach is more cost-effective but may require more initial investment in infrastructure and expertise.
- **Performance Requirements**: OpenAI's models often provide superior performance, especially for complex tasks.
- **Data Privacy**: The open-source approach offers more control over data handling, which may be crucial for sensitive applications.
- **Scalability**: Both approaches can scale, but OpenAI's solution may be easier to scale quickly.
- **Customization Needs**: Open-source models offer more flexibility for fine-tuning and adaptation to specific domains.

> **Note**: It's essential to evaluate the trade-offs between cost, performance, and control when deciding between these approaches. In some cases, a hybrid solution utilizing aspects of both methods might be optimal.

In [24]:
# Initialize the OpenAIEmbeddings model with specified parameters
# 'text-embedding-3-large' is the model name and 3072 is the dimensionality of the embeddings
openai_embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large", dimensions=3072
)

# Generate embeddings for the unofficial occupations
# This converts each occupation name in the unofficial_occupations list into a high-dimensional vector
embeddings_unofficial_occupations_openai = openai_embeddings_model.embed_documents(
    unofficial_occupations
)

# Generate embeddings for the official occupations
# This converts each occupation name in the official_occupations list into a high-dimensional vector
embeddings_official_occupations_openai = openai_embeddings_model.embed_documents(
    official_occupations
)

# Create a dictionary mapping each unofficial occupation name to its corresponding embedding vector
corpus_unofficial_occupations_openai = {
    t: np.array(embeddings_unofficial_occupations_openai[i])
    for i, t in enumerate(unofficial_occupations)
}

# Create a dictionary mapping each official occupation name to its corresponding embedding vector
corpus_official_occupations_openai = {
    t: np.array(embeddings_official_occupations_openai[i])
    for i, t in enumerate(official_occupations)
}

In [25]:
corpus_official_occupations_openai["pedreiro"]

array([-0.03269531,  0.0567728 , -0.00798459, ..., -0.01239539,
       -0.01652234, -0.01168937])

In [26]:
corpus_unofficial_occupations_openai["motorista"]

array([-0.00596753,  0.00174657,  0.00824405, ..., -0.00579513,
        0.00595186, -0.00164959])

In [27]:
# Save the unofficial occupations embeddings dictionary to a file using joblib
# This allows for efficient storage and later retrieval of the embeddings
joblib.dump(
    corpus_unofficial_occupations_openai,
    "outputs/occupations/corpus_unofficial_occupations_openai.joblib",
)

# Save the official occupations embeddings dictionary to a file using joblib
# This allows for efficient storage and later retrieval of the embeddings
joblib.dump(
    corpus_official_occupations_openai,
    "outputs/occupations/corpus_official_occupations_openai.joblib",
)

['outputs/occupations/corpus_official_occupations_openai.joblib']

In [28]:
# Load the official occupations embeddings dictionary from a file using joblib
# This retrieves the precomputed embeddings for official occupations
corpus_official_occupations_openai = joblib.load(
    "outputs/occupations/corpus_official_occupations_openai.joblib"
)

# Load the unofficial occupations embeddings dictionary from a file using joblib
# This retrieves the precomputed embeddings for unofficial occupations
corpus_unofficial_occupations_openai = joblib.load(
    "outputs/occupations/corpus_unofficial_occupations_openai.joblib"
)

In [29]:
# Initialize the OllamaEmbeddings model with the specified model name 'bge-m3'
# This model will be used to generate embeddings for text data
ollama_embeddings_model = OllamaEmbeddings(
    model="bge-m3",
)

In [30]:
# Generate embeddings for the unofficial occupations using the Ollama embeddings model
# This converts each occupation name in the unofficial_occupations list into a high-dimensional vector
embeddings_unofficial_occupations_ollama = ollama_embeddings_model.embed_documents(
    unofficial_occupations
)

# Generate embeddings for the official occupations using the Ollama embeddings model
# This converts each occupation name in the official_occupations list into a high-dimensional vector
embeddings_official_occupations_ollama = ollama_embeddings_model.embed_documents(
    official_occupations
)

# Create a dictionary mapping each unofficial occupation name to its corresponding embedding vector
corpus_unofficial_occupations_ollama = {
    t: np.array(embeddings_unofficial_occupations_ollama[i])
    for i, t in enumerate(unofficial_occupations)
}

# Create a dictionary mapping each official occupation name to its corresponding embedding vector
corpus_official_occupations_ollama = {
    t: np.array(embeddings_official_occupations_ollama[i])
    for i, t in enumerate(official_occupations)
}

In [31]:
corpus_official_occupations_ollama["pedreiro"]

array([ 0.01889289,  0.02634712, -0.03264777, ..., -0.02796427,
       -0.05883273,  0.02549875])

In [32]:
corpus_unofficial_occupations_ollama["motorista"]

array([-0.00576592, -0.00856881,  0.01095934, ..., -0.00442688,
       -0.11000092, -0.00187536])

In [33]:
# Save the unofficial occupations embeddings dictionary generated by the Ollama model to a file using joblib
# This allows for efficient storage and later retrieval of the embeddings
joblib.dump(
    corpus_unofficial_occupations_ollama,
    "outputs/occupations/corpus_unofficial_occupations_ollama.joblib",
)

# Save the official occupations embeddings dictionary generated by the Ollama model to a file using joblib
# This allows for efficient storage and later retrieval of the embeddings
joblib.dump(
    corpus_official_occupations_ollama,
    "outputs/occupations/corpus_official_occupations_ollama.joblib",
)

['outputs/occupations/corpus_official_occupations_ollama.joblib']

In [34]:
# Load the official occupations embeddings dictionary generated by the Ollama model from a file using joblib
# This retrieves the precomputed embeddings for official occupations
corpus_official_occupations_ollama = joblib.load(
    "outputs/occupations/corpus_official_occupations_ollama.joblib"
)

# Load the unofficial occupations embeddings dictionary generated by the Ollama model from a file using joblib
# This retrieves the precomputed embeddings for unofficial occupations
corpus_unofficial_occupations_ollama = joblib.load(
    "outputs/occupations/corpus_unofficial_occupations_ollama.joblib"
)

In this section, we will execute the first step of our process: **matching unofficial occupation names to the most relevant professions in the official IBGE classification**. This involves two main tasks: calculating semantic similarity and performing zero-shot learning.

## Step 1: Semantic Similarity Calculation

To begin, we need to find the closest matching profession for each unofficial occupation name. This is achieved by calculating the semantic similarity between the input occupation name and each profession in the IBGE classification. Here's how we will approach this task:

1. **Embedding the Input Occupation Name**:
    - **Concept**: An *embedding* is a numerical representation of words or phrases in a high-dimensional vector space. Embeddings capture semantic meaning by positioning similar concepts closer together.
    - **Process**: The unofficial occupation name is transformed into an embedding. This is typically done using techniques from natural language processing (NLP), where each word or phrase is converted into a vector.

2. **Comparing with IBGE Classifications**:
    - **Embedding the Professions**: Every profession in the IBGE classification is also represented by an embedding.
    - **Similarity Calculation**: Once the unofficial occupation name is embedded, its vector is compared with the vectors of all IBGE professions. A common approach to measure similarity is the *cosine similarity*.        
    - **Distance Metrics**: Alternative techniques might use other distance measures (e.g., Euclidean distance), but cosine similarity is often preferred because it focuses on the angle between vectors rather than their magnitude.

3. **Identifying Closest Matches**:
    - By determining which IBGE profession embeddings are closest to the input occupation name embedding, we can narrow down the search space to the most relevant matches. This step is crucial for ensuring that we focus on the most likely candidates for the next phase of our process.

In [35]:
def find_most_similar_names(
    input_text: str,
    unofficial_occupations: dict[str, np.ndarray],
    official_occupations: dict[str, np.ndarray],
    top_n: int = 10,
) -> list[str]:
    """
    Finds the most similar names to a given target name based on cosine similarity.

    Args:
        input_text (str): The name for which to find similar names.
        unofficial_occupations (dict[str, np.ndarray]): A dictionary mapping names to their unofficial embedding vectors.
        official_occupations (dict[str, np.ndarray]): A dictionary mapping names to their official embedding vectors.
        top_n (int, optional): The number of most similar names to return. Defaults to 10.

    Returns:
        list[str]: A list of the most similar names.
    """
    # Retrieve the embedding vector for the target name from the unofficial_occupations dictionary
    target_embedding = unofficial_occupations[input_text]

    # Calculate cosine similarity between the target name's embedding and all names in official_occupations
    similarity_scores = {
        name: cosine_similarity([target_embedding], [official_occupations[name]])[0][0]
        for name in official_occupations
    }

    # Sort the names based on similarity scores in descending order and select the top 'n' names
    most_similar_names = sorted(
        similarity_scores, key=similarity_scores.get, reverse=True
    )[:top_n]

    # Return the list of most similar names
    return most_similar_names


def get_random_occupation_name(occupation_embeddings: dict[str, np.ndarray]) -> str:
    """
    Chooses a random occupation name from the occupation_embeddings dictionary.

    Args:
        occupation_embeddings (dict[str, np.ndarray]): A dictionary mapping occupation names to their embedding vectors.

    Returns:
        str: A randomly chosen occupation name.
    """
    # Choose a random occupation name from the list of occupation names
    random_occupation_name = np.random.choice(list(occupation_embeddings.keys()))

    # Return the randomly chosen occupation name
    return random_occupation_name

In [36]:
random_occupation_name = get_random_occupation_name(
    corpus_unofficial_occupations_openai
)
random_occupation_name

'aux de cobranças'

In [37]:
random_occupation_candidates_openai = find_most_similar_names(
    random_occupation_name,
    corpus_unofficial_occupations_openai,
    corpus_official_occupations_openai,
    top_n=10,
)
random_occupation_candidates_openai

['supervisor de cobrança',
 'operador de cobrança bancária',
 'gerente de crédito e cobrança',
 'supervisor de crédito e cobrança',
 'analista de cobrança (instituições financeiras)',
 'auxiliar de faturamento',
 'gerente de recuperação de crédito',
 'tabelião de protestos',
 'supervisor de contas a pagar',
 'diretor de recuperação de créditos em operações de intermediação financeira']

In [38]:
random_occupation_candidates_ollama = find_most_similar_names(
    random_occupation_name,
    corpus_unofficial_occupations_ollama,
    corpus_official_occupations_ollama,
    top_n=10,
)
random_occupation_candidates_ollama

['supervisor de cobrança',
 'operador de cobrança bancária',
 'cobrador externo',
 'analista de cobrança (instituições financeiras)',
 'gerente de crédito e cobrança',
 'auxiliar de faturamento',
 'cobrador interno',
 'supervisor de contas a pagar',
 'supervisor de crédito e cobrança',
 'prensista de compensados']

### Step 1.2: Reranking models

We can use reranking models to improve the accuracy of our search results. Reranking models, also known as cross-encoders, significantly improve the accuracy of search results by reordering documents in terms of their relevance to a given query. This section will discuss the role and importance of rerankers in search systems and how they compare to other models.

We could, for instance, completely avoid going to the second stage of the search process if the reranker is confident enough that the first result is the best one. This would save us a lot of time, as we would not need to go through the zero-shot learning process at all.


#### What is a Reranking Model?

A reranking model is a type of model that takes a query and a document pair and outputs a similarity score. This score is then used to reorder the documents by their relevance to the query.
The process typically involves two steps:

1. **Initial Retrieval (Bi-Encoder Stage):**
   - An embedding model quickly retrieves a set of potentially relevant documents from a larger dataset.
   - During this step, document vectors are pre-computed. Given a query $ q $, a similarity score with document vectors $ d_i $ is computed using a function such as cosine similarity:
     
     $$
     \text{sim}(q, d_i) = \frac{q \cdot d_i}{\|q\| \cdot \|d_i\|}
     $$
     
2. **Reranking (Cross-Encoder Stage):**
   - The reranker receives the query and candidate documents, and then it re-evaluates each candidate in real time.
   - Instead of relying on pre-computed vectors, the reranker processes the query and each document together to generate a new, more precise similarity score.

> **Note:** Rerankers are generally slower due to the need for full transformer computations on each query-document pair, but they offer better accuracy by incorporating the specific context of the query.

#### Why Use Rerankers?

Despite their slower performance, rerankers are much more accurate than embedding models. Here are some key reasons:

- **Information Compression in Bi-encoders**: Bi-encoders compress all possible meanings of a document into a single vector, leading to information loss. This can reduce the accuracy of the retrieval process.

- **Contextual Understanding in Rerankers**: Rerankers analyze the document's meaning specific to the user query at the time of the query, reducing information loss and improving relevance.

#### Comparison: Bi-encoders vs. Rerankers

| Feature | Bi-encoders | Rerankers |
|---------|-------------|-----------|
| **Computation Time** | Fast (pre-computed) | Slow (real-time) |
| **Information Loss** | High (single vector) | Low (raw information) |
| **Contextual Understanding** | Limited (pre-query) | High (query-specific) |

#### How Rerankers Work

When using bi-encoder models with vector search, the heavy transformer computation occurs during the creation of the initial vectors (our Step 1). This means:

1. **Initial Vector Creation**: Pre-compute document vectors using transformers.
2. **Query Processing**:
- Run a single transformer computation to create the query vector.
- Compare the query vector to document vectors using cosine similarity or another lightweight metric.

In contrast, rerankers do not pre-compute vectors. Instead:

1. **Query Processing**:
- Feed the query and a single document into the transformer.
- Run a full transformer inference step.
- Output a similarity score for that document.

<p align="center">
<img src="images/rerankers.webp" alt="Rerankers Diagram" style="width: 80%; height: 80%"/>
</p>


We'll now use the [BGE-Reranker-v2-M3](https://huggingface.co/BAAI/bge-reranker-v2-m3) model to refine our search results and improve the accuracy of our occupation-profession matching process.

In [39]:
# Define the path to the reranker model
reranker_path = "BAAI/bge-reranker-v2-m3"

# Load the tokenizer for the reranker model from the specified path
reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_path)

# Load the reranker model for sequence classification from the specified path
reranker = AutoModelForSequenceClassification.from_pretrained(reranker_path)

# Set the reranker model to evaluation mode
# This disables dropout and other training-specific behaviors
reranker.eval()

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(8194, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, o

In [40]:
def rerank_occupation_candidates(
    reranker_model: PreTrainedModel,
    reranker_tokenizer: PreTrainedTokenizer,
    target_occupation: str,
    candidate_occupations: list[str],
    top_n: int = 5,
) -> list[str]:
    """
    Reranks a list of occupation candidates based on a reranker model.

    Args:
        reranker_model (PreTrainedModel): The reranker model to use for reranking.
        reranker_tokenizer (PreTrainedTokenizer): The tokenizer for the reranker model.
        target_occupation (str): The occupation name for which candidates are being reranked.
        candidate_occupations (list[str]): The list of occupation candidates to rerank.
        top_n (int, optional): The number of top candidates to return. Defaults to 5.

    Returns:
        list[str]: The reranked list of occupation candidates.
    """

    # Create pairs of target occupation and each candidate for reranking
    query_candidate_pairs = [
        [f'Qual a profissão mais parecida com "{target_occupation}?', candidate]
        for candidate in candidate_occupations
    ]

    # Use the reranker model to rerank the occupation candidates
    with torch.no_grad():
        # Tokenize the pairs for input to the reranker model
        model_inputs = reranker_tokenizer(
            query_candidate_pairs,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=512,
        )

        # Get the logits (scores) from the reranker model
        logits = (
            reranker_model(**model_inputs, return_dict=True)
            .logits.view(
                -1,
            )
            .float()
        )

        # Sort the candidates based on the scores in descending order
        sorted_indices = logits.argsort(descending=True)
        sorted_candidates = [query_candidate_pairs[i][1] for i in sorted_indices]

    # Return the top 'n' reranked candidates
    return sorted_candidates[:top_n]

In [41]:
rerank_occupation_candidates(
    reranker_model=reranker,
    reranker_tokenizer=reranker_tokenizer,
    target_occupation=random_occupation_name,
    candidate_occupations=random_occupation_candidates_openai,
    top_n=5,
)

['supervisor de cobrança',
 'auxiliar de faturamento',
 'operador de cobrança bancária',
 'analista de cobrança (instituições financeiras)',
 'supervisor de crédito e cobrança']

Reranking is a important step in the search process that can significantly improve the relevance and quality of search results. Here's how we can exploit reranking effectively:

1. **Early Satisfaction of Results**
    - If the reranker produces highly satisfactory results, we may choose to conclude the search process at this stage.
    - This approach can save considerable time and computational resources.

2. **Filtering Irrelevant Results**
    - The reranker can act as a powerful filter, eliminating less relevant or off-topic results.
    - This filtration process creates a more focused dataset for subsequent steps.

3. **Resource Optimization**
    - By potentially bypassing the zero-shot learning (ZSL) stage, we can allocate resources more efficiently.
    - This is particularly beneficial in time-sensitive scenarios or when dealing with large-scale data processing.

> **Note**: The decision to use reranking alone or in combination with ZSL should be based on careful analysis of your specific use case, data characteristics, and performance requirements.


If we feel that the reranking is not enough, we can move to the next step: Zero-shot classification. Let's see.

## Step 2: Zero-Shot Classification

After identifying the closest matches through semantic similarity, the next step is to classify the occupation names into professions using zero-shot classification. But what exactly is zero-shot classification, and how does it apply to our occupation classification task?

### Understanding Zero-Shot Classification

Zero-shot classification is a powerful machine classification technique that enables models to classify data they haven't explicitly seen during training. This capability is particularly valuable in our occupation classification task, where we aim to categorize numerous potentially novel occupation names.

**Key Characteristics of Zero-Shot Classification:**
- Classifies unseen data points without specific training examples
- Leverages broader knowledge and generalization capabilities
- Particularly useful for handling new, unexpected inputs

> **Comparison with related techniques:**
> - **Few-Shot Classification:** Model learns from a small number of examples per class
> - **One-Shot Classification:** Model learns from a single example per class
> - **Traditional Supervised Classification:** Model learns from labeled training data.


### The Role of Emergence in Large Language Models

Zero-shot classification in Large Language Models (LLMs) is closely tied to the concept of emergence, a phenomenon where complex systems exhibit unexpected properties or behaviors that arise from simpler components.

**Emergence in Various Systems:**
- **Ant Colonies:** Individual ants following simple rules create complex, organized structures
- **Neural Networks:** Basic artificial neurons combine to produce sophisticated cognitive abilities
- **LLMs:** Vast networks of parameters give rise to unexpected capabilities like zero-shot classification

In LLMs, emergence manifests as the model's ability to perform tasks it wasn't explicitly trained on, such as our occupation classification task.
> To know more about the concept of emergence, consider [watching this video](https://www.youtube.com/watch?v=16W7c0mb-rE).

### Applying Zero-Shot Classification to Occupation Classification

Let's break down how we use zero-shot classification in our specific task:

1. **Model Preparation:**
    - Instead of training on every possible occupation name, the model is pre-trained on a diverse corpus of text data
    - This broad knowledge base allows the model to understand language patterns, context, and relationships between concepts

2. **Classification Process:**
    - When presented with a new occupation name, the model:
    - Analyzes the semantic content of the name
    - Compares it to its learned knowledge of professions and job roles
    - Predicts the most appropriate IBGE classification based on semantic similarity

3. **Utilizing Semantic Similarity:**
    - The model uses the semantic similarity scores calculated in the previous step to inform its classification decisions
    - This allows for nuanced understanding of occupation names that might be synonyms or closely related to known professions

4. **Handling Ambiguity:**
    - For occupation names that could fit multiple categories, the model may provide a ranked list of potential classifications
    - This approach acknowledges the innate complexity of occupation categorization

### Advantages and Considerations

**Benefits of Zero-Shot Classification in This Context:**
- Flexibility to classify previously unseen occupation names
- Ability to adapt to evolving job markets and new professions
- Reduced need for constant model retraining as new occupations emerge

**Potential Challenges:**
- Accuracy may vary for highly specialized or region-specific occupations
- The model's performance is dependent on the quality and breadth of its pre-training data
- Misclassifications can occur if the model lacks contextual understanding for certain occupations

Let's use ZSL to create a flexible and powerful occupation classification system capable of handling the diverse and evolving landscape of job titles.

In [42]:
prompt_zsl = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um especialista em classificação de profissões com conhecimento profundo do Código Brasileiro de Ocupações (CBO). "
            "Sua tarefa é classificar uma profissão de entrada em uma das profissões candidatas fornecidas ou determinar que não há correspondência adequada. "
            "Siga estas diretrizes:"
            "\n\n1. Analise cuidadosamente a profissão de entrada e compare-a com a lista de profissões candidatas."
            "\n2. Se houver uma correspondência viável, classifique a entrada na profissão candidata mais apropriada."
            "\n3. Se não houver nenhuma correspondência minimamente viável, retorne NENHUMA."
            "\n4. Considere sinônimos, variações de nomenclatura e descrições similares ao fazer a classificação."
            "\n5. Use seu conhecimento para determinar a correspondência mais precisa, mesmo que não haja uma correspondência exata."
            "\n\nLembre-se: sua precisão na classificação é crucial para o sucesso desta tarefa.",
        ),
        ("human", "Profissão de entrada: {text}"),
    ]
)

In [43]:
model_openai = ChatOpenAI(
    model="gpt-4o-mini",  # 'model' specifies the model to use, in this case 'gpt-4o-mini'
    temperature=0,  # 'temperature' controls the randomness of the model's output, with 0 being more deterministic
    timeout=30,  # 'timeout' specifies the maximum time (in seconds) to wait for a response from the model
)

model_llama = ChatOllama(
    model="llama3.1",  # 'model' specifies the model to use, in this case 'llama3.1'
    temperature=0,  # 'temperature' controls the randomness of the model's output, with 0 being more deterministic
    base_url="http://localhost:11434",  # 'base_url' specifies the base URL for the model's API endpoint
    timeout=30,  # 'timeout' specifies the maximum time (in seconds) to wait for a response from the model
)

In [44]:
random_occupation_name = get_random_occupation_name(
    corpus_unofficial_occupations_openai
)
random_occupation_candidates_ollama = find_most_similar_names(
    random_occupation_name,
    corpus_unofficial_occupations_ollama,
    corpus_official_occupations_ollama,
    top_n=10,
)
random_occupation_candidates_openai = find_most_similar_names(
    random_occupation_name,
    corpus_unofficial_occupations_openai,
    corpus_official_occupations_openai,
    top_n=10,
)

# Rerank the occupation candidates using the reranker model

reranked_candidates_ollama = rerank_occupation_candidates(
    reranker_model=reranker,
    reranker_tokenizer=reranker_tokenizer,
    target_occupation=random_occupation_name,
    candidate_occupations=random_occupation_candidates_ollama,
    top_n=5,
)

reranked_candidates_openai = rerank_occupation_candidates(
    reranker_model=reranker,
    reranker_tokenizer=reranker_tokenizer,
    target_occupation=random_occupation_name,
    candidate_occupations=random_occupation_candidates_openai,
    top_n=5,
)

print(f"Occupation: {random_occupation_name}")
print("Reranked candidates (Ollama):", reranked_candidates_ollama)
print("Reranked candidates (OpenAI):", reranked_candidates_openai)

Occupation: motorista  de aplicativo uber
Reranked candidates (Ollama): ['condutor de veículos a pedais', 'motorista de táxi', 'motorista de ônibus urbano', 'motorista de carro de passeio', 'motorista de furgão ou veículo similar']
Reranked candidates (OpenAI): ['motorista de táxi', 'motorista de ônibus urbano', 'motorista de carro de passeio', 'motorista de furgão ou veículo similar', 'ajudante de motorista']


In [45]:
# Let's sanity-check what was discarded by the reranker
print(
    "Discarded candidates (Ollama):",
    [
        c
        for c in random_occupation_candidates_ollama
        if c not in reranked_candidates_ollama
    ],
)
print(
    "Discarded candidates (OpenAI):",
    [
        c
        for c in random_occupation_candidates_openai
        if c not in reranked_candidates_openai
    ],
)

Discarded candidates (Ollama): ['ajudante de motorista', 'motorista de ônibus rodoviário', 'motorista de trólebus', 'motorista operacional de guincho', 'piloto de competição automobilística']
Discarded candidates (OpenAI): ['motorista de trólebus', 'motorista de ônibus rodoviário', 'motorista operacional de guincho', 'motorista de caminhão (rotas regionais e internacionais)', 'motociclista no transporte de documentos e pequenos volumes']


In [46]:
reranked_candidates_openai

['motorista de táxi',
 'motorista de ônibus urbano',
 'motorista de carro de passeio',
 'motorista de furgão ou veículo similar',
 'ajudante de motorista']

In [47]:
# Let's try the model with a prompt


class ProfissaoOpenAI(BaseModel):
    """Converte uma profissão não descrita com CBO para uma profissão descrita com CBO (Classificação Brasileira de Ocupações)"""

    profissao_sem_cbo: str = Field(..., description="Profissão não descrita com CBO")
    profissao_com_cbo: Literal[tuple(reranked_candidates_openai)] = Field(
        ...,
        description=f"Profissão descrita com CBO. Pode ser uma das seguintes: {reranked_candidates_openai}",
    )


class ProfissaoOllama(BaseModel):
    """Converte uma profissão não descrita com CBO para uma profissão descrita com CBO (Classificação Brasileira de Ocupações)"""

    profissao_sem_cbo: str = Field(..., description="Profissão não descrita com CBO")
    profissao_com_cbo: Literal[tuple(reranked_candidates_ollama)] = Field(
        ...,
        description=f"Profissão descrita com CBO. Pode ser uma das seguintes: {reranked_candidates_ollama}",
    )

    # Note that some information from the model is passed to the LLM. I've written those parts in Portuguese to prime the model to answer also in Portuguese.
    # In the class above, everything in Portuguese is passed to the model. I've kept the rest in English, so you can know what will be passed to the model.


zsl_ollama = prompt_zsl | model_llama.with_structured_output(
    ProfissaoOllama, include_raw=True
)
zsl_openai = prompt_zsl | model_openai.with_structured_output(
    ProfissaoOpenAI, include_raw=True
).with_retry(stop_after_attempt=5)

In [48]:
result_zsl_ollama = zsl_ollama.invoke(random_occupation_name)
result_zsl_openai = zsl_openai.invoke(random_occupation_name)

In [49]:
result_zsl_ollama

{'raw': AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-03-12T19:03:30.732077902Z', 'done': True, 'done_reason': 'stop', 'total_duration': 546910272, 'load_duration': 21100078, 'prompt_eval_count': 512, 'prompt_eval_duration': 28000000, 'eval_count': 48, 'eval_duration': 493000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-f7eeb99f-2226-4ca4-9583-3dc5aa8ea185-0', tool_calls=[{'name': 'ProfissaoOllama', 'args': {'profissao_com_cbo': 'motorista de carro de passeio', 'profissao_sem_cbo': 'motorista de aplicativo uber'}, 'id': '8536c02b-d498-42f5-b3b9-d7c08487d619', 'type': 'tool_call'}], usage_metadata={'input_tokens': 512, 'output_tokens': 48, 'total_tokens': 560}),
 'parsed': ProfissaoOllama(profissao_sem_cbo='motorista de aplicativo uber', profissao_com_cbo='motorista de carro de passeio'),
 'parsing_error': None}

In [50]:
result_zsl_openai

{'raw': AIMessage(content='{"profissao_sem_cbo":"motorista de aplicativo uber","profissao_com_cbo":"motorista de carro de passeio"}', additional_kwargs={'parsed': ProfissaoOpenAI(profissao_sem_cbo='motorista de aplicativo uber', profissao_com_cbo='motorista de carro de passeio'), 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 486, 'total_tokens': 513, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-d0179389-60ba-4d1b-b986-d894d1b3e5f4-0', usage_metadata={'input_tokens': 486, 'output_tokens': 27, 'total_tokens': 513, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'parsed': ProfissaoOpenAI(prof

In [51]:
result_zsl_ollama["parsed"].profissao_com_cbo

'motorista de carro de passeio'

In [52]:
result_zsl_openai["parsed"].profissao_com_cbo

'motorista de carro de passeio'

In [53]:
type(model_llama), type(model_openai)

(langchain_ollama.chat_models.ChatOllama,
 langchain_openai.chat_models.base.ChatOpenAI)

In [54]:
# Let's generalize the process to a function


def classify_occupation_zero_shot(
    occupation_name: str,
    occupation_candidates: list[str],
    language_model: Any,
    prompt_template: Any,
    include_raw: bool = False,
) -> Any:
    """
    Classifies an occupation name into a predefined list of occupation candidates using a zero-shot learning approach.

    Args:
        occupation_name (str): The name of the occupation to classify.
        occupation_candidates (list[str]): A list of predefined occupation candidates.
        language_model (Any): The language model to use for classification.
        prompt_template (Any): The prompt template to use with the language model.
        include_raw (bool, optional): Whether to include the raw response from the language model. Defaults to False.

    Returns:
        Any: The response from the language model, including the classified occupation.
    """
    # Add 'NENHUMA' (none) to the list of occupation candidates
    occupation_candidates.append("NENHUMA")

    # Define a Pydantic model for the classification response
    class ProfissaoDataModel(BaseModel):
        """Converte uma profissão não descrita com CBO para uma profissão descrita com CBO (Classificação Brasileira de Ocupações)"""

        ocupacao_sem_cbo: str = Field(..., description="Profissão não descrita com CBO")
        ocupacao_com_cbo: Literal[tuple(occupation_candidates)] = Field(
            ...,
            description=f"Profissão descrita com CBO que melhor descreve a ocupação sem cbo. Pode ser uma das seguintes: {occupation_candidates}",
        )

    # Note that some information from the model is passed to the LLM. I've written those parts in Portuguese to prime the model to answer also in Portuguese.
    # In the class above, everything in Portuguese is passed to the model. I've kept the rest in English, so you can know what will be passed to the model.

    # Create a runnable pipeline with the prompt template and language model
    classification_pipeline = prompt_template | language_model.with_structured_output(
        ProfissaoDataModel, include_raw=include_raw
    ).with_retry(
        wait_exponential_jitter=True,  # Add jitter to the exponential backoff
        stop_after_attempt=4,  # Try at most 4 times
    )

    # Invoke the pipeline with the occupation name to get the classification response
    response = classification_pipeline.invoke(occupation_name)

    return response

In [55]:
# Let's test the function with a new occupation

random_occupation_name = get_random_occupation_name(
    corpus_unofficial_occupations_openai
)
random_occupation_candidates_ollama = find_most_similar_names(
    random_occupation_name,
    corpus_unofficial_occupations_ollama,
    corpus_official_occupations_ollama,
    top_n=10,
)
random_occupation_candidates_openai = find_most_similar_names(
    random_occupation_name,
    corpus_unofficial_occupations_openai,
    corpus_official_occupations_openai,
    top_n=10,
)

# Rerank the occupation candidates using the reranker model

reranked_candidates_ollama = rerank_occupation_candidates(
    reranker_model=reranker,
    reranker_tokenizer=reranker_tokenizer,
    target_occupation=random_occupation_name,
    candidate_occupations=random_occupation_candidates_ollama,
    top_n=5,
)

reranked_candidates_openai = rerank_occupation_candidates(
    reranker_model=reranker,
    reranker_tokenizer=reranker_tokenizer,
    target_occupation=random_occupation_name,
    candidate_occupations=random_occupation_candidates_openai,
    top_n=5,
)

In [56]:
print(f"Occupation: {random_occupation_name}")
print("Reranked candidates (Ollama):", reranked_candidates_ollama)
print("Reranked candidates (OpenAI):", reranked_candidates_openai)

Occupation: maceneiro
Reranked candidates (Ollama): ['marceneiro', 'macheiro, a máquina', 'maquetista na marcenaria', 'macheiro, a mão', 'marcheteiro']
Reranked candidates (OpenAI): ['marceneiro', 'macheiro, a máquina', 'carpinteiro', 'operador de maçaroqueira', 'pedreiro']


In [57]:
# Classify the occupation using the zero-shot learning approach with the Ollama model
result_zsl_ollama = classify_occupation_zero_shot(
    occupation_name=random_occupation_name,
    occupation_candidates=reranked_candidates_ollama,
    language_model=model_llama,
    prompt_template=prompt_zsl,
    include_raw=False,
)

result_zsl_ollama

ProfissaoDataModel(ocupacao_sem_cbo='maceneiro', ocupacao_com_cbo='marceneiro')

In [58]:
# Classify the occupation using the zero-shot learning approach with the OpenAI model
result_zsl_openai = classify_occupation_zero_shot(
    occupation_name=random_occupation_name,
    occupation_candidates=reranked_candidates_openai,
    language_model=model_openai,
    prompt_template=prompt_zsl,
    include_raw=False,
)
result_zsl_openai

ProfissaoDataModel(ocupacao_sem_cbo='maceneiro', ocupacao_com_cbo='marceneiro')

### Analyzing Zero-Shot Learning Results

Since we're using random examples, let's check the results of our zero-shot classification when I ran it on `atendente de consultorio odontologico`.

#### Model Outputs
- **OpenAI's Model Output:**
```python
ProfissaoDataModel(
ocupacao_sem_cbo='atendente de consultorio odontologico',
ocupacao_com_cbo='recepcionista de consultório médico ou dentário'
)
```
- **Llama 3.1 Output:**
```python
ProfissaoDataModel(
ocupacao_sem_cbo='atendente de consultorio odontologico',
ocupacao_com_cbo='atendente de enfermagem'
)
```

The results show that the two models provided different classifications for the occupation "atendente de consultorio odontologico." This discrepancy highlights the importance of evaluating model outputs and considering the context and domain-specific knowledge when interpreting results.

#### Comparison of Model Capabilities
- **OpenAI Model:**
    - Identified the occupation as **"recepcionista de consultório médico ou dentário"**, which directly relates to a dental office setting.

- **Llama 3.1 Model:**
    - Classified the occupation as **"atendente de enfermagem"**, which typically associates more broadly with healthcare and may not specifically align with a dental office.

#### Interpretation and Context
- The **OpenAI model** seems to provide a more contextually accurate classification, likely due to its extensive training on a larger and more diverse dataset.
- The **Llama 3.1 model** offers a broader interpretation, which might suggest a more generic healthcare role.

#### Model Evaluation and Cost Considerations

Given the assumption that larger models trained on more extensive datasets (like OpenAI's) tend to yield better results, it's important to also consider the associated costs.

- **Embedding Model**:
    - OpenAI Embedding Model (`text-embedding-3-large`) costs 0.13 USD per 1,000,000 tokens. For our task, encoding the names of occupations is negligible in cost.

- **Language Model**:
    - OpenAI LLM `gpt-4o-mini` costs 0.150 USD per 1,000,000 tokens of input and 0.60 USD per 1,000,000 tokens of output. Each request includes:
        - Occupation name
        - List of candidate occupations
        - Schema for the Pydantic model
        - Custom prompt created for classification

This can become expensive, depending on the number of requests required.

#### Estimating Classification Costs
To estimate the cost of classifying our dataset, consider the following points:
- Number of occupations to classify.
- Average token count per request (including input and output).

Let's calculate the average cost for classifying a single occupation name using the OpenAI LLM `gpt-4o-mini` model.

In [59]:
# Classify the occupation using the zero-shot learning approach with the OpenAI model
result_zsl_openai = classify_occupation_zero_shot(
    occupation_name=random_occupation_name,
    occupation_candidates=reranked_candidates_openai,
    language_model=model_openai,
    prompt_template=prompt_zsl,
    include_raw=True,  # Include the raw response from the model, which contains the tokens used in the call
)

price_of_call_input = (0.150 / 1000000) * result_zsl_openai["raw"].usage_metadata[
    "input_tokens"
]
price_of_call_output = (0.60 / 1000000) * result_zsl_openai["raw"].usage_metadata[
    "output_tokens"
]
estimated_price = (price_of_call_input + price_of_call_output) * len(
    corpus_unofficial_occupations_ollama
)

print(
    f"Estimated price for classifying all {len(corpus_unofficial_occupations_ollama)} occupations: ${estimated_price:.2f}"
)

Estimated price for classifying all 3941 occupations: $0.34


`Less than 50 cents for the entire dataset is not a bad deal!` Let's proceed with the classification process.

In [60]:
def full_zero_shot_occupation_classification(
    input_occupation: str,
    reranker_model: PreTrainedModel,
    reranker_tokenizer: PreTrainedTokenizer,
    language_model: Any,
    prompt_template: Any,
    unofficial_occupation_embeddings: dict[str, np.ndarray],
    official_occupation_embeddings: dict[str, np.ndarray],
    include_raw_output: bool = False,
    initial_candidate_count: int = 10,
    final_candidate_count: int = 5,
) -> dict[str, Any]:
    """
    Performs a full zero-shot learning pipeline for occupation classification.

    Args:
        input_occupation (str): The occupation name to classify.
        reranker_model (PreTrainedModel): The model used for reranking candidates.
        reranker_tokenizer (PreTrainedTokenizer): The tokenizer for the reranker model.
        language_model (Any): The language model used for zero-shot classification.
        prompt_template (Any): The template for generating prompts.
        unofficial_occupation_embeddings (dict[str, np.ndarray]): Embeddings of unofficial occupation names.
        official_occupation_embeddings (dict[str, np.ndarray]): Embeddings of official occupation names.
        include_raw_output (bool, optional): Whether to include raw model output. Defaults to False.
        initial_candidate_count (int, optional): Number of initial candidates to consider. Defaults to 10.
        final_candidate_count (int, optional): Number of final candidates after reranking. Defaults to 5.

    Returns:
        dict[str, Any]: The classification result.
    """

    # Step 1: Find the most similar occupation names
    similar_occupation_candidates = find_most_similar_names(
        input_occupation,
        unofficial_occupation_embeddings,
        official_occupation_embeddings,
        top_n=initial_candidate_count,
    )

    # Step 2: Rerank the occupation candidates using the reranker model
    reranked_occupation_candidates = rerank_occupation_candidates(
        reranker_model=reranker_model,
        reranker_tokenizer=reranker_tokenizer,
        target_occupation=input_occupation,
        candidate_occupations=similar_occupation_candidates,
        top_n=final_candidate_count,
    )

    # Step 3: Classify the occupation using zero-shot learning
    zero_shot_classification_result = classify_occupation_zero_shot(
        occupation_name=input_occupation,
        occupation_candidates=reranked_occupation_candidates,
        language_model=language_model,
        prompt_template=prompt_template,
        include_raw=include_raw_output,
    )

    return zero_shot_classification_result

In [61]:
full_zero_shot_occupation_classification(
    input_occupation=random_occupation_name,
    reranker_model=reranker,
    reranker_tokenizer=reranker_tokenizer,
    language_model=model_openai,
    prompt_template=prompt_zsl,
    unofficial_occupation_embeddings=corpus_unofficial_occupations_openai,
    official_occupation_embeddings=corpus_official_occupations_openai,
    include_raw_output=False,
    initial_candidate_count=10,
    final_candidate_count=5,
)

ProfissaoDataModel(ocupacao_sem_cbo='maceneiro', ocupacao_com_cbo='marceneiro')

In [62]:
def wrapper_zsl(input_occupation: str, model_name: str):
    if model_name == "openai":
        model = model_openai
        unofficial_occupation_embeddings = corpus_unofficial_occupations_openai
        official_occupation_embeddings = corpus_official_occupations_openai
    elif model_name == "ollama":
        model = model_llama
        unofficial_occupation_embeddings = corpus_unofficial_occupations_ollama
        official_occupation_embeddings = corpus_official_occupations_ollama
    else:
        raise ValueError("Invalid model name. Choose either 'openai' or 'ollama'.")
    try:
        return full_zero_shot_occupation_classification(
            input_occupation=input_occupation,
            reranker_model=reranker,
            reranker_tokenizer=reranker_tokenizer,
            language_model=model,
            prompt_template=prompt_zsl,
            unofficial_occupation_embeddings=unofficial_occupation_embeddings,
            official_occupation_embeddings=official_occupation_embeddings,
            include_raw_output=False,
            initial_candidate_count=15,
            final_candidate_count=5,
        ).ocupacao_com_cbo
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [63]:
wrapper_zsl("cabista", "ollama")

'cableador'

In [64]:
wrapper_zsl("cabista", "openai")

'cableador'

In [65]:
unofficial_data_dedup

,id,code,text,text_split_1,text_split_2
3,2561,NaN,aposentado,desempregado,None
4,2562,NaN,agricultora,agricultora,None
6,2564,NaN,auxiliar de creche,auxiliar de creche,None
7,2565,NaN,tecnico judiciário,tecnico judiciário,None
13,2571,NaN,técnico em enfermágem,técnico em enfermágem,None
...,...,...,...,...,...
8241,10857,NaN,pessoa com deficiência,pessoa com deficiência,None
8243,10859,NaN,agricultor de subsistência,agricultor de subsistência,None
8245,10861,NaN,técnico de segurança eletrônica,técnico de segurança eletrônica,None
8247,10863,NaN,coordenador de vendas,coordenador de vendas,None


In [66]:
unofficial_data_dedup["ollama_labels"] = unofficial_data_dedup["text_split_1"].apply(
    lambda x: wrapper_zsl(x, "ollama")
)

An error occurred: 1 validation error for ProfissaoDataModel
ocupacao_com_cbo
  Input should be 'ajudante de motorista', 'engenheiro de aplicativos em computação', 'motorista de táxi', 'motorista de carro de passeio', 'condutor de veículos a pedais' or 'NENHUMA' [type=literal_error, input_value='engenheiro de aplicativos em computadora', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error
An error occurred: 1 validation error for ProfissaoDataModel
ocupacao_com_cbo
  Input should be 'assistente de vendas', 'vendedor pracista', 'comerciante varejista', 'vendedor em comércio atacadista', 'vendedor ambulante' or 'NENHUMA' [type=literal_error, input_value='vendedora', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error
An error occurred: 1 validation error for ProfissaoDataModel
ocupacao_com_cbo
  Input should be 'autônomo', 'representante comercial autônomo', 'ajudante de motorista', 'instrutor d

In [67]:
unofficial_data_dedup["openai_labels"] = unofficial_data_dedup["text_split_1"].apply(
    lambda x: wrapper_zsl(x, "openai")
)

In [68]:
unofficial_data_dedup["openai_labels"]

3                                            desempregado
4                       trabalhador agropecuário em geral
6                    auxiliar de desenvolvimento infantil
7                                  auxiliar de judiciário
13                                  técnico de enfermagem
                              ...                        
8241                                              NENHUMA
8243                      produtor agropecuário, em geral
8245    instalador de sistemas eletroeletrônicos de se...
8247                       supervisor de vendas comercial
8249                                    servente de obras
Name: openai_labels, Length: 3941, dtype: object

In [69]:
unofficial_data_dedup.sample(20)

,id,code,text,text_split_1,text_split_2,ollama_labels,openai_labels
5974,8590,NaN,marido de aluguel,marido de aluguel,None,empregado doméstico diarista,NENHUMA
3743,6356,NaN,motorista de transportes urbanos,motorista de transportes urbanos,None,motorista de ônibus urbano,motorista de ônibus urbano
1916,4482,NaN,torneiro mecânico,torneiro mecânico,None,técnico mecânico,NENHUMA
4907,7523,NaN,operadora de negócios,operadora de negócios,None,operador de negócios,operador de negócios
4986,7602,NaN,penionista,penionista,None,gerente de pensão,gerente de pensão
3933,6547,NaN,vaqueiro + pescador,vaqueiro + pescador,None,pescador profissional,NENHUMA
3976,6590,NaN,não sei,não sei,None,NENHUMA,NENHUMA
6211,8827,NaN,bicos servente,bicos servente,None,None,NENHUMA
4503,7119,NaN,tecnico em panificação,tecnico em panificação,None,None,"operador de forno (fabricação de pães, biscoit..."
741,3302,NaN,estofador,estofador,None,estofador de móveis,estofador de móveis


In [70]:
unofficial_data_dedup.to_parquet("outputs/occupations/unofficial_data_dedup.parquet")

In [71]:
agreements = (
    unofficial_data_dedup["openai_labels"] == unofficial_data_dedup["ollama_labels"]
).sum() / len(unofficial_data_dedup)
print(f"Agreement between OpenAI and Ollama: {agreements:.2%}")

Agreement between OpenAI and Ollama: 43.47%


## Takeaways
- **Effective Occupation Classification**: Combining NLP and ML techniques can effectively address the complex problem of occupation classification, ensuring consistency and accuracy in data.

- **Importance of Semantic Similarity**: Analyzing semantic similarity between occupation names and standardized professions is crucial for narrowing down the search space and improving classification accuracy.

- **Utility of Zero-shot Classification**: Zero-shot learning is a powerful tool for handling diverse and evolving job titles, reducing the need for constant retraining.

- **Role of Reranking Models**: Reranking models significantly enhance the relevance and quality of search results by providing query-specific context, which can sometimes eliminate the need for further processing.

- **Cost-Performance Balance**: Choosing between open-source and commercial models involves trade-offs between cost, performance, and control over data handling, necessitating careful evaluation based on specific project requirements.

# Questions

1. What is the primary objective of the notebook described in the content?

2. Which standardized classification does the notebook use as a reference for occupations?

3. What are some of the key challenges in automated occupation classification mentioned in the notebook?

4. Which two main approaches for generating embeddings are compared in the notebook, and what models are used for each?

5. How is semantic similarity calculated between occupation names and standardized professions in this notebook?

6. What role does zero-shot classification play in the occupation classification system described?

7. How do reranking models (or cross-encoders) improve the search results in the classification pipeline?

8. What data preprocessing techniques are applied to the unofficial and official occupation datasets before classification?

9. How does the notebook address cost considerations when using commercial models such as OpenAI's LLM and embedding models?

10. Describe the full pipeline applied in the notebook that combines semantic similarity, reranking, and zero-shot classification for occupation classification.


`Answers are commented inside this cell`
<!--
1. The primary objective of the notebook is to automatically map each user-input occupation name (unofficial) to the closest matching standardized profession from the Brazilian IBGE classification, which comprises 2,557 professions.
2. The notebook uses the IBGE (Brazilian Institute of Geography and Statistics) classification as the standard reference for occupations.
3. Some key challenges mentioned include semantic variability, ambiguity in occupation names, the lack of sufficient labeled data, the need for generalization to unseen occupations, and striking a balance between efficiency and accuracy.
4. The notebook compares an open-source approach and a commercial approach. The open-source method employs the BGE-M3 model (for both embeddings and language modeling via Llama 3.1), while the commercial method uses OpenAI’s models, specifically the "text-embedding-3-large" for embeddings and GPT-based models (e.g., gpt-4o-mini) for zero-shot classification.
5. Semantic similarity is calculated using cosine similarity between the embedding vectors of occupation names and those of standardized professions. The formula used is the cosine of the angle between the two vectors, computed as the dot product divided by the product of their magnitudes.
6. Zero-shot classification enables the model to classify occupation names into professions without having explicit training examples for each category. This allows the system to handle previously unseen or diverse occupation names by utilizing its general language understanding.
7. Reranking models (cross-encoders) take the initial candidate list produced by the semantic similarity search and re-evaluate the query-document pairs in real time. This improves accuracy by incorporating query-specific context, filtering out less relevant candidates, and potentially allowing the system to stop the search process early if a highly confident match is found.
8. The data preprocessing includes converting text to lowercase, splitting strings (using the '-' character to separate parts), removing noise (such as unnecessary segments), deduplicating entries by removing duplicate names, and standardizing common terms (e.g., replacing various forms of “desempregado” or "autônomo").
9. The notebook discusses cost considerations by comparing the expenses of using commercial models, where each API call (embedding and inference) incurs cost based on token usage. For instance, using the OpenAI LLM (gpt-4o-mini) and OpenAI embedding model, the notebook estimates that classifying all occupation names in the dataset costs less than 50 cents, while also considering the trade-offs between performance, accuracy, and control over data.
10. The full pipeline starts by embedding both unofficial and official occupation names to calculate semantic similarity (using cosine similarity) and retrieve an initial set of candidate matches. It then refines these candidates with a reranking step using a cross-encoder model (BGE-Reranker-v2-M3). Finally, zero-shot classification is applied via language models (either OpenAI’s or Llama 3.1) using a custom prompt and a defined Pydantic schema to classify the input occupation into the most appropriate standardized profession.
-->